In [ ]:
import matplotlib.pyplot as plt
import glob
from radar import RadarMesure
from scipy.signal import find_peaks
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

from scipy.optimize import curve_fit



def bin_to_dist(x,pad_factor=1):
    c = 299792458
    B = 2e9   #bandwith
    ramptime = 102400e-9
    frequence_echantillonage = 10e6 
    rangebinresolution = frequence_echantillonage/1024              #DF
    fb = x*rangebinresolution
    dist = fb*c*ramptime/(2*B)
    dist=dist/pad_factor
    return(dist)

    
# Chemin vers le fichier
data = glob.glob('/home/nicolas/Documents/Dku-toolbox/data/newcod/17GHz_*.txt')
data.sort()


def compute_ccalib(dataset):
    
    c = 299792458
    lambdaa = c/(17e9)
    sigmacr = (np.pi*4*0.3302**4)/(3*lambdaa**2)

    def func(R, c):
        return (c * sigmacr / (R**4))

    # Création de l'instance
    ri = []
    [ri.append(RadarMesure(i,beta = 8, pad_factor = 5)) for i in dataset]

    pic = []

    for i in ri :
        df = i.df
        peaks,prop = find_peaks(df.copol.loc[0.4:5],height=1e-6)
        if len(peaks) != 0 :
            pic.append((bin_to_dist(peaks[0],5)+0.4,prop['peak_heights'][0]**2))
    dfpic = pd.DataFrame(pic).sort_values(by=[0]) 

    X = np.asarray(dfpic[0])
    y = np.asarray(dfpic[1])

    # Fit du modèle
    popt, pcov = curve_fit(func, X, y)
    y_pred = func(X, *popt)

    # Calcul du R²
    r2 = r2_score(y, y_pred)

    # Erreur sur les paramètres
    perr = np.sqrt(np.diag(pcov))
    c_err = perr[0] if len(perr) > 0 else np.nan

    return {
        "c_calib": popt[0],
        "c_err": c_err,
        "r2": r2,
        }


compute_ccalib(data)


[WARNING] Mois 7 non reconnu dans leg_dict
[WARNING] Mois 7 non reconnu dans leg_dict
[WARNING] Mois 7 non reconnu dans leg_dict
[WARNING] Mois 7 non reconnu dans leg_dict
[WARNING] Mois 7 non reconnu dans leg_dict
[WARNING] Mois 7 non reconnu dans leg_dict
[WARNING] Mois 7 non reconnu dans leg_dict
[WARNING] Mois 7 non reconnu dans leg_dict
[WARNING] Mois 7 non reconnu dans leg_dict
[WARNING] Mois 7 non reconnu dans leg_dict
[WARNING] Mois 7 non reconnu dans leg_dict
[WARNING] Mois 7 non reconnu dans leg_dict
[WARNING] Mois 7 non reconnu dans leg_dict
[WARNING] Mois 7 non reconnu dans leg_dict
[WARNING] Mois 7 non reconnu dans leg_dict
[WARNING] Mois 7 non reconnu dans leg_dict
           0             1
7   1.779045  1.436901e-11
8   1.868983  1.397070e-11
9   1.958921  1.287294e-11
10  2.048859  1.101346e-11
11  2.138796  9.510828e-12
12  2.228734  8.082940e-12
13  2.318672  7.702411e-12
14  2.408609  7.375351e-12
15  2.498547  7.032622e-12
0   2.588485  6.843582e-12
1   2.678423  6

/tmp/ipykernel_24083/3404890889.py:39: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(c * sigmacr / (R**4))


{'c_calib': np.float64(1.351554009532748e-23),
 'c_err': np.float64(5.73469316110254e-25),
 'r2': 0.9330627435682957}